In [4]:
!pip install git+https://github.com/openai/CLIP.git



huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-au1dzdla
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-au1dzdla
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done


In [ ]:
import os
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import clip
from transformers import BlipProcessor, BlipForConditionalGeneration
import numpy as np
from tqdm import tqdm

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

class TinyImageNet(Dataset):
    def __init__(self, root, split='val', transform=None):
        self.root = os.path.expanduser(root)
        self.transform = transform
        self.split = split
        
        if split == 'train':
            self.image_paths = []
            self.labels = []
            
            # Process train directory which has subdirectories by class
            train_dir = os.path.join(self.root, 'train')
            self.classes = sorted(os.listdir(train_dir))
            self.class_to_idx = {cls_name: i for i, cls_name in enumerate(self.classes)}
            
            for class_name in self.classes:
                class_dir = os.path.join(train_dir, class_name, 'images')
                if not os.path.isdir(class_dir):
                    continue
                    
                for img_name in os.listdir(class_dir):
                    if img_name.endswith('.JPEG'):
                        self.image_paths.append(os.path.join(class_dir, img_name))
                        self.labels.append(self.class_to_idx[class_name])
                        
        elif split == 'val':
            self.image_paths = []
            self.labels = []
            
            # Process val directory
            val_dir = os.path.join(self.root, 'val')
            images_dir = os.path.join(val_dir, 'images')
            
            # Read val annotations
            val_annotations_file = os.path.join(val_dir, 'val_annotations.txt')
            self.classes = []
            self.class_to_idx = {}
            
            with open(val_annotations_file, 'r') as f:
                for line in f:
                    parts = line.strip().split('\t')
                    img_name, class_id = parts[0], parts[1]
                    
                    if class_id not in self.class_to_idx:
                        self.classes.append(class_id)
                        self.class_to_idx[class_id] = len(self.classes) - 1
                    
                    self.image_paths.append(os.path.join(images_dir, img_name))
                    self.labels.append(self.class_to_idx[class_id])

    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        label = self.labels[idx]
        
        img = Image.open(img_path).convert('RGB')
        
        if self.transform:
            img = self.transform(img)
            
        return img, label

def get_classnames(dataset, words_file_path):
    """Get formatted class names from dataset using WordNet mappings."""
    if hasattr(dataset, 'classes'):
        # Load WordNet ID to name mapping
        wordnet_map = {}
        with open(words_file_path, 'r') as f:
            for line in f:
                parts = line.strip().split('\t')
                if len(parts) == 2:
                    wordnet_id, names = parts
                    # Take the first name if there are multiple comma-separated ones
                    name = names.split(',')[0].strip()
                    wordnet_map[wordnet_id] = name
        
        # Replace WordNet IDs with human-readable names
        classnames = []
        for c in dataset.classes:
            if c in wordnet_map:
                classnames.append(wordnet_map[c])
            else:
                # Try without 'n' prefix if not found directly
                if c.startswith('n') and c[1:] in wordnet_map:
                    classnames.append(wordnet_map[c[1:]])
                else:
                    # Fallback to the original ID
                    classnames.append(c)
        
        return classnames
    return None

def prepare_dataset(root_dir):
    """Prepare and return the Tiny ImageNet dataset with appropriate transforms."""
    # Basic transform without normalization for dataset loading
    basic_transform = transforms.Compose([
        transforms.Resize(224, interpolation=transforms.InterpolationMode.BICUBIC),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
    ])
    
    # Create dataset instance
    val_dataset = TinyImageNet(root_dir, split='val', transform=basic_transform)
    val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False, num_workers=4)
    
    return val_dataset, val_loader
    
def generate_captions(blip_model, blip_processor, images, prompts):
    """Generate captions for the given images using BLIP model."""
    all_captions = []
    
    # Create PIL images from tensors
    pil_images = []
    for img in images:
        # Convert tensor to PIL image
        # Ensure values are within 0-1 range
        np_img = img.cpu().numpy().transpose(1, 2, 0)
        np_img = np.clip(np_img, 0, 1)
        pil_img = Image.fromarray((np_img * 255).astype(np.uint8))
        pil_images.append(pil_img)
    
    # Process images with BLIP
    batch_size = len(pil_images)
    with torch.no_grad():
        for prompt in prompts:
            batch_captions = []
            
            # Process each image individually
            for i in range(batch_size):
                inputs = blip_processor(images=pil_images[i], text=prompt, return_tensors="pt").to(device)
                generated_ids = blip_model.generate(**inputs, max_length=50)
                caption = blip_processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
                batch_captions.append(caption)
                
            all_captions.append(batch_captions)
    
    return all_captions


def compute_image_embeddings(clip_model, images):
    """Compute CLIP image embeddings with proper normalization."""
    # Apply CLIP-specific normalization
    clip_mean = torch.tensor((0.48145466, 0.4578275, 0.40821073), device=images.device).view(1, 3, 1, 1)
    clip_std = torch.tensor((0.26862954, 0.26130258, 0.27577711), device=images.device).view(1, 3, 1, 1)
    normalized_images = (images - clip_mean) / clip_std
    
    with torch.no_grad():
        image_features = clip_model.encode_image(normalized_images)
        image_features = image_features / image_features.norm(dim=1, keepdim=True)
    return image_features

def compute_text_embeddings(clip_model, texts):
    """Compute CLIP text embeddings."""
    with torch.no_grad():
        text_features = clip_model.encode_text(texts)
        text_features = text_features / text_features.norm(dim=1, keepdim=True)
    return text_features

def image_caption_encoding(image_probs, caption_probs, K=5, xi=0.08, epsilon=1e-12):
    """
    Implement ICE method as described in the paper.
    
    Args:
        image_probs: Probability distribution from image embeddings
        caption_probs: Probability distribution from caption embeddings
        K: Number of top classes to consider
        xi: Scaling factor for lambda
        epsilon: Small constant to avoid division by zero
        
    Returns:
        Final probability distribution after ICE
    """
    # Find top K classes based on image probabilities
    topk_values, topk_indices = torch.topk(image_probs, k=K, dim=1)
    
    # Extract probabilities for top K classes
    image_topk_probs = torch.gather(image_probs, 1, topk_indices)
    caption_topk_probs = torch.gather(caption_probs, 1, topk_indices)
    
    # Compute standard deviation for confidence selection
    image_std = torch.std(image_topk_probs, dim=1, keepdim=True)
    caption_std = torch.std(caption_topk_probs, dim=1, keepdim=True)
    
    # Compute lambda based on equation 3 in the paper
    norm = torch.maximum(torch.norm(torch.cat([image_std, caption_std], dim=1), dim=1, keepdim=True), 
                         torch.tensor(epsilon, device=device))
    
    lambda_values = xi * (caption_std / norm)
    
    # Combine image and caption probabilities
    combined_topk_probs = image_topk_probs + lambda_values * caption_topk_probs
    
    # Create new probability distribution
    output_probs = torch.zeros_like(image_probs)
    output_probs.scatter_(1, topk_indices, combined_topk_probs)
    
    return output_probs

def evaluate_model(clip_model, blip_model, blip_processor, val_loader, classnames, caption_prompts):
    """Evaluate the model on the validation set."""
    text_inputs = torch.cat([clip.tokenize(f"a photo of a {c}") for c in classnames]).to(device)
    text_features = compute_text_embeddings(clip_model, text_inputs)
    
    total = 0
    correct_base = 0
    correct_ice = 0
    
    # Print sample outputs for the first few batches only
    print_samples = 200
    sample_count = 0
    
    for batch_idx, (images, labels) in enumerate(tqdm(val_loader)):
        images = images.to(device)
        labels = labels.to(device)
        batch_size = images.shape[0]
        total += batch_size
        
        # Compute image embeddings and probabilities
        image_features = compute_image_embeddings(clip_model, images)
        image_logits = 100.0 * image_features @ text_features.T
        image_probs = F.softmax(image_logits, dim=1)
        
        # Generate captions using BLIP
        batch_captions = generate_captions(blip_model, blip_processor, images, caption_prompts)
        
        # Process captions
        caption_features_list = []
        for prompt_captions in batch_captions:
            caption_tokens = clip.tokenize(prompt_captions).to(device)
            caption_features = compute_text_embeddings(clip_model, caption_tokens)
            caption_features_list.append(caption_features)
        
        # Compute centroid of caption embeddings
        caption_features = torch.stack(caption_features_list).mean(dim=0)
        
        # Compute caption probabilities
        caption_logits = 100.0 * caption_features @ text_features.T
        caption_probs = F.softmax(caption_logits, dim=1)
        
        # Apply ICE
        ice_probs = image_caption_encoding(image_probs, caption_probs)
        
        # Compute predictions
        _, base_preds = torch.max(image_probs, 1)
        _, ice_preds = torch.max(ice_probs, 1)
        
        # Update metrics
        correct_base += (base_preds == labels).sum().item()
        correct_ice += (ice_preds == labels).sum().item()
        
        # Print examples for debugging
        if batch_idx < print_samples:
            print(f"\n--- Batch {batch_idx} examples ---")
            # Print top 5 examples from the batch
            for i in range(min(5, batch_size)):
                true_label = labels[i].item()
                base_pred = base_preds[i].item()
                ice_pred = ice_preds[i].item()
                
                print(f"\nImage {sample_count + i}:")
                print(f"  True class: {classnames[true_label]}")
                print(f"  CLIP prediction: {classnames[base_pred]}")
                print(f"  ICE prediction: {classnames[ice_pred]}")
                
                # Print captions from all prompts
                print("  Captions:")
                for j, prompt in enumerate(caption_prompts):
                    print(f"    {prompt}: {batch_captions[j][i]}")
                
                # Print confidence scores for predictions
                base_conf = image_probs[i, base_pred].item() * 100
                ice_conf = ice_probs[i, ice_pred].item() * 100
                print(f"  CLIP confidence: {base_conf:.2f}%")
                print(f"  ICE confidence: {ice_conf:.2f}%")
            
            sample_count += batch_size
    
    base_acc = 100 * correct_base / total
    ice_acc = 100 * correct_ice / total
    
    return base_acc, ice_acc

def main():
    # Path to Tiny ImageNet dataset
    tiny_imagenet_path = '/kaggle/input/tiny-imagenet/tiny-imagenet-200'
    
    # Load models
    print("Loading CLIP model...")
    clip_model, clip_preprocess = clip.load("ViT-B/32", device=device)
    clip_model.eval()
    
    print("Loading BLIP model...")
    blip_processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
    blip_model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base").to(device)
    blip_model.eval()
    
    # Prepare dataset
    print("Preparing dataset...")
    val_dataset, val_loader = prepare_dataset(tiny_imagenet_path)
    classnames = get_classnames(val_dataset, '/kaggle/input/tiny-imagenet/tiny-imagenet-200/words.txt')  
    print("\nSample class mappings:")
    for i, c in enumerate(val_dataset.classes[:20]):
        print(f"{c} -> {classnames[i]}")
    # Set caption prompts
    caption_prompts = ["a photo of", "a picture of", "a photo containing"]
    
    # Evaluate
    print("Evaluating...")
    base_acc, ice_acc = evaluate_model(
        clip_model, blip_model, blip_processor, val_loader, classnames, caption_prompts
    )
    
    print(f"Base CLIP Accuracy: {base_acc:.2f}%")
    print(f"ICE Accuracy: {ice_acc:.2f}%")
    print(f"Improvement: {ice_acc - base_acc:.2f}%")

if __name__ == "__main__":
    main()

Using device: cuda
Loading CLIP model...
Loading BLIP model...
Preparing dataset...

Sample class mappings:
n03444034 -> go-kart
n04067472 -> reel
n04070727 -> refrigerator
n02808440 -> bathtub
n04399382 -> teddy
n04179913 -> sewing machine
n02823428 -> beer bottle
n04146614 -> school bus
n02226429 -> grasshopper
n04371430 -> swimming trunks
n07753592 -> banana
n03770439 -> miniskirt
n02056570 -> king penguin
n02906734 -> broom
n02125311 -> cougar
n04486054 -> triumphal arch
n04285008 -> sports car
n03763968 -> military uniform
n03814639 -> neck brace
n03837869 -> obelisk
Evaluating...


  0%|          | 0/157 [00:00<?, ?it/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(tru


--- Batch 0 examples ---

Image 0:
  True class: go-kart
  CLIP prediction: go-kart
  ICE prediction: go-kart
  Captions:
    a photo of: a photo of a group of people sitting on the ground
    a picture of: a picture of a group of people sitting on the ground
    a photo containing: a photo containing of a group of people in a street
  CLIP confidence: 87.30%
  ICE confidence: 87.30%

Image 1:
  True class: reel
  CLIP prediction: lifeboat
  ICE prediction: lifeboat
  Captions:
    a photo of: a photo of a sheep
    a picture of: a picture of a sheep
    a photo containing: a photo containing the image of a sheep
  CLIP confidence: 31.74%
  ICE confidence: 31.74%

Image 2:
  True class: refrigerator
  CLIP prediction: beacon
  ICE prediction: beacon
  Captions:
    a photo of: a photo of a city with a clock
    a picture of: a picture of a city with a lot of buildings
    a photo containing: a photo containing the city of london taken from a window in the tower
  CLIP confidence: 10.8

  1%|▏         | 2/157 [00:57<1:14:33, 28.86s/it]


--- Batch 1 examples ---

Image 64:
  True class: fountain
  CLIP prediction: fountain
  ICE prediction: fountain
  Captions:
    a photo of: a photo of the fountain at night
    a picture of: a picture of a fountain with lights on it
    a photo containing: a photo containing the lights of the space shuttle
  CLIP confidence: 33.81%
  ICE confidence: 41.58%

Image 65:
  True class: water tower
  CLIP prediction: water tower
  ICE prediction: water tower
  Captions:
    a photo of: a photo of a field with trees and a building in the background
    a picture of: a picture of a field with trees and a building in the background
    a photo containing: a photo containing of the site of the proposed new school
  CLIP confidence: 47.39%
  ICE confidence: 47.39%

Image 66:
  True class: maypole
  CLIP prediction: maypole
  ICE prediction: maypole
  Captions:
    a photo of: a photo of a group of people standing around a tent
    a picture of: a picture of a group of people standing around a 

...


--- Batch 42 examples ---

Image 2688:
  True class: bison
  CLIP prediction: bison
  ICE prediction: bison
  Captions:
    a photo of: a photo of a bear that is laying down
    a picture of: a picture of a dog that is laying down
    a photo containing: a photo containing of a bear that was found in the wild
  CLIP confidence: 40.16%
  ICE confidence: 40.16%

Image 2689:
  True class: basketball
  CLIP prediction: basketball
  ICE prediction: basketball
  Captions:
    a photo of: a photo of a basketball game with the ball in the air
    a picture of: a picture of a basketball game with the ball in the air
    a photo containing: a photo containing the basketball game between the two teams
  CLIP confidence: 71.24%
  ICE confidence: 77.78%

Image 2690:
  True class: picket fence
  CLIP prediction: picket fence
  ICE prediction: picket fence
  Captions:
    a photo of: a photo of a fence in the fog
    a picture of: a picture of a fence with a sky background
    a photo containing: a 

 28%|██▊       | 44/157 [21:48<55:28, 29.45s/it]


--- Batch 43 examples ---

Image 2752:
  True class: lifeboat
  CLIP prediction: lifeboat
  ICE prediction: lifeboat
  Captions:
    a photo of: a photo of a boat in the water
    a picture of: a picture of a boat in the water
    a photo containing: a photo containing of a boat in the water
  CLIP confidence: 99.71%
  ICE confidence: 99.71%

Image 2753:
  True class: African elephant
  CLIP prediction: African elephant
  ICE prediction: African elephant
  Captions:
    a photo of: a photo of a baby elephant in a dirt field
    a picture of: a picture of a small town with a small elephant
    a photo containing: a photo containing the elephant in the village
  CLIP confidence: 90.48%
  ICE confidence: 96.48%

Image 2754:
  True class: beer bottle
  CLIP prediction: beer bottle
  ICE prediction: beer bottle
  Captions:
    a photo of: a photo of a dog
    a picture of: a picture of a dog
    a photo containing: a photo containing of a dog
  CLIP confidence: 92.38%
  ICE confidence: 92.

 29%|██▊       | 45/157 [22:18<55:18, 29.63s/it]


--- Batch 44 examples ---

Image 2816:
  True class: torch
  CLIP prediction: torch
  ICE prediction: torch
  Captions:
    a photo of: a photo of a man holding a frth
    a picture of: a picture of a man holding a frth
    a photo containing: a photo containing a man holding a frc
  CLIP confidence: 35.57%
  ICE confidence: 35.57%

Image 2817:
  True class: golden retriever
  CLIP prediction: boa constrictor
  ICE prediction: boa constrictor
  Captions:
    a photo of: a photo of a dog laying in the snow
    a picture of: a picture of a dog laying in the snow
    a photo containing: a photo containing of a dog laying in the snow
  CLIP confidence: 33.40%
  ICE confidence: 33.40%

Image 2818:
  True class: bannister
  CLIP prediction: bannister
  ICE prediction: bannister
  Captions:
    a photo of: a photo of a building with a lot of windows
    a picture of: a picture of a city with buildings and a sky
    a photo containing: a photo containing with the image of a man in a suit
  CL